我们在这章讨论字典和集合，因为它们背后都是哈希表，下面是本章的大纲

- 常用字典方法
- 特别处理遗失的键
- 在标准库中，dict 的变化
- set 与 frozenset 形态
- 哈希表的工作原理
- 哈希表的影响（键形态限制，无法预知的排序等等）

## 什么是可哈希化

如果一个对象有一个哈希值，而且在生命周期中不被改变（它需要实现一个 `__hash__()` 方法），而且可以与其它对象比较（需要实现 `__eq__()` 方法），就是可哈希化的。看下面例子

In [2]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

In [5]:
t1 = (1, 2, [30, 40]) # 其中列表是可变的，所以没有哈希值
hash(t1)

TypeError: unhashable type: 'list'

In [6]:
tf = (1, 2, frozenset([30, 40])) #frozenset 是冻结的集合，不可变的，所以有哈希值
hash(tf)

-4118419923444501110

## 构建字典方法

In [8]:
a = dict(one = 1, two = 2, three = 3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})
a == b == c == d == e

True

除了常规语法以及 dict 构建之外，我们可以使用字典生成式来构建字典，dictcomp 会由任何一个可迭代对象产生一对 key:value 来构建 dict，下面是使用字典生成式的一个例子:

In [11]:
DIAL_CODES = [
    (86, 'China'),
    (91, 'India'),
    (1, 'United States')
]

country_code = {country: code for code, country in DIAL_CODES}
country_code

{'China': 86, 'India': 91, 'United States': 1}

字典有一个内置方法 `d.update(m, [**kargs])` 它会先判断 m，如果 m 有 keys 方法，就假定它是个映射，负责迭代 m，假设它的元素是 (key, value)，这也就是说明任何产生 (key, value) 的迭代对象都可以构建字典

## 使用 setdefault 处理找不到的键

当 dict 使用 d[k] 时，发现 k 不是现有键时，会跑出 KeyError 错误，我们知道 d.get(k, default) 是 d[k] 的另一种用法，使用预处理值要比处理 KeyError 异常更方便，但是如果你要更新你找到的值，使用 `__getitem__()` 和 `get()` 都很没有效率，我们下面先看一下一般时候处理找不到键的方式:

In [15]:
#!/usr/bin/env python
# encoding: utf-8
import sys
import re

WORD_RE = re.compile('\w+') # \w 是匹配任意字母或数字，+ 是匹配一次到任意次

index = {}
#with open(sys.argv[1], encoding="utf-8") as fp:  #正常文件名是参数传的
with open("/home/kaka/test.txt", encoding="utf-8") as fp:
    for line_no, line in enumerate(fp, 1):    # line_no 是索引（从 1 开始），line 是行的内容
        for match in WORD_RE.finditer(line):  # 返回所有匹配子串，返回类型是迭代器
            word = match.group()              # group 获取该单词 （match 是一个对象）
            column_no = match.start() + 1     # 获取列数，索引从 0 开始
            location = (line_no, column_no)   # 构造一个元组，内容是 (row, col)
            # 这样写很糟糕，这里仅仅是为了演示
            occurrences = index.get(word, []) # 判断该单词是否被添加过，没有返回 [ ]，注意返回的是原列表的一个备份
            occurrences.append(location)      # 为该 key 对应的值添加内容
            index[word] = occurrences         # 这要搜索 word 这个 key 第二次
for word in sorted(index, key = str.upper):   # 按照字母顺序排序，忽略大小写
    print(word, index[word])        

a [(2, 49)]
aided [(2, 194)]
and [(2, 181)]
art [(2, 96)]
artistic [(2, 17), (2, 135)]
but [(2, 56)]
channeled [(2, 122)]
computer [(2, 185)]
design [(2, 200)]
film [(2, 175)]
fine [(2, 91)]
had [(2, 6)]
I [(2, 1), (2, 43), (2, 60), (2, 117)]
impulses [(2, 144)]
in [(2, 85)]
Instead [(2, 108)]
invested [(2, 71)]
kid [(2, 51)]
mainly [(2, 153)]
much [(2, 80)]
music [(2, 168)]
my [(2, 88), (2, 132)]
My [(1, 1)]
never [(2, 65)]
point [(1, 13)]
since [(2, 37)]
skills [(2, 100)]
starting [(1, 4)]
strong [(2, 10)]
tendencies [(2, 26)]
through [(2, 160)]
ve [(2, 3), (2, 62), (2, 119)]
was [(2, 45)]


我们处理 occurrences 的三行可以使用 dict.setdefault 来改为一行

In [16]:
#!/usr/bin/env python
# encoding: utf-8
import sys
import re

WORD_RE = re.compile('\w+') # \w 是匹配任意字母或数字，+ 是匹配一次到任意次

index = {}
#with open(sys.argv[1], encoding="utf-8") as fp:  #正常文件名是参数传的
with open("/home/kaka/test.txt", encoding="utf-8") as fp:
    for line_no, line in enumerate(fp, 1):    # line_no 是索引（从 1 开始），line 是行的内容
        for match in WORD_RE.finditer(line):  # 返回所有匹配子串，返回类型是迭代器
            word = match.group()              # group 获取该单词 （match 是一个对象）
            column_no = match.start() + 1     # 获取列数，索引从 0 开始
            location = (line_no, column_no)   # 构造一个元组，内容是 (row, col)
            index.setdefault(word, []).append(location) #如果没有 word 这个 key，设为 [ ]，setdefault 会传回该值，所以不用二次搜索就可以被更新
for word in sorted(index, key = str.upper):   # 按照字母顺序排序，忽略大小写
    print(word, index[word])      

a [(2, 49)]
aided [(2, 194)]
and [(2, 181)]
art [(2, 96)]
artistic [(2, 17), (2, 135)]
but [(2, 56)]
channeled [(2, 122)]
computer [(2, 185)]
design [(2, 200)]
film [(2, 175)]
fine [(2, 91)]
had [(2, 6)]
I [(2, 1), (2, 43), (2, 60), (2, 117)]
impulses [(2, 144)]
in [(2, 85)]
Instead [(2, 108)]
invested [(2, 71)]
kid [(2, 51)]
mainly [(2, 153)]
much [(2, 80)]
music [(2, 168)]
my [(2, 88), (2, 132)]
My [(1, 1)]
never [(2, 65)]
point [(1, 13)]
since [(2, 37)]
skills [(2, 100)]
starting [(1, 4)]
strong [(2, 10)]
tendencies [(2, 26)]
through [(2, 160)]
ve [(2, 3), (2, 62), (2, 119)]
was [(2, 45)]


换句话说 `index.setdefault(word, []).append(location)` 与下面等价

In [18]:
#if key not in my_dict:
#    my_dict[key] = []
#else:
#    my_dict[key].append(new_value)

## 可弹性查找键的映射

有时在找不到键时，返回一些虚构值还是很好的，这里有两种方法，第一种是使用预设字典而不是普通字典，第二种是将字典变成子类别或其它的映射类型，并添加一个 `__missing__` 方法，下面会讨论两种做法

### defaultdict 找不到键的另一种做法

下面是使用 collections.defaultdict 优雅的解决上面搜索不存在的键的问题。

In [20]:
#!/usr/bin/env python
# encoding: utf-8
import sys
import re
import collections

WORD_RE = re.compile('\w+') 

index = collections.defaultdict(list)  # 使用 list 建立 defaultdict，将它当成 default_factory
#with open(sys.argv[1], encoding="utf-8") as fp:  
with open("/home/kaka/test.txt", encoding="utf-8") as fp:
    for line_no, line in enumerate(fp, 1):    
        for match in WORD_RE.finditer(line):  
            word = match.group()              
            column_no = match.start() + 1    
            location = (line_no, column_no)  
            # 如果不存在 word 键，会调用初始化传的 default_factory 产生一个预设值, 如果没有指定 default_factory，会产生 KeyError 异常
            index[word].append(location) 
for word in sorted(index, key = str.upper):  
    print(word, index[word])      

a [(2, 49)]
aided [(2, 194)]
and [(2, 181)]
art [(2, 96)]
artistic [(2, 17), (2, 135)]
but [(2, 56)]
channeled [(2, 122)]
computer [(2, 185)]
design [(2, 200)]
film [(2, 175)]
fine [(2, 91)]
had [(2, 6)]
I [(2, 1), (2, 43), (2, 60), (2, 117)]
impulses [(2, 144)]
in [(2, 85)]
Instead [(2, 108)]
invested [(2, 71)]
kid [(2, 51)]
mainly [(2, 153)]
much [(2, 80)]
music [(2, 168)]
my [(2, 88), (2, 132)]
My [(1, 1)]
never [(2, 65)]
point [(1, 13)]
since [(2, 37)]
skills [(2, 100)]
starting [(1, 4)]
strong [(2, 10)]
tendencies [(2, 26)]
through [(2, 160)]
ve [(2, 3), (2, 62), (2, 119)]
was [(2, 45)]


工作原理: 当我们初始化一个 defaultdict 时候，要提供一个方法，当 `__getitem__()` 找不到键的时候，会用它产生一个预设值，这里我们将 list 传进去，每次调用会产生一个空列表。

注意：defaultdict 的 default_factory 只是为了提供一个预设值，不供其它方法使用，例如 dd 是 defaultdict，k 是不存在的键， dd[k] 会调用 default_factory 产生一个预设值，而 dd.get(k) 仍然传回 None，下面是例子

In [23]:
import collections

index = collections.defaultdict(list) 
print(index.get('hello'))

None


调用 default_factory 的 defaultdict 机制，实际上是调用 `__missing__()` 特殊方法，这是所有变暗映射类型都有的方法

## `__missing__` 方法

映射处理找不到键时，底层使用的是 `__missing__()` 方法，基础的 dict 并没有这个方法，但是如果你将 dict 变成子类别，并提供一个 `__missing__()` 方法，标准的 `dict.__getitem__()`  会在找不到键时调用它，而非发出 KeyError

`__missing__()` 只被 `__getitem__()` 调用（即 d[k] 运算符），`__missing__()` 方法的存在，并不会影响到其它方法查询键的行为，例如 get 或 `__contains__`（in 运算符），这就是 defaultdict 只能与 `__getitem__()` 一起使用的原因

如果你想要一个映射，它的键在查询时会被转成 str 

In [16]:
class StrKeyDict0(dict):
    def __missing__(self, key):
        # 判断是不是 str 类型，是的话没有此 key 直接抛出异常，不是的话转成 str 重新调用本方法
        # 如果没有 isinstance 判断会不断调用自身，死循环
        if isinstance(key, str): # 判断类型，type()不会认为子类是一种父类类型。isinstance()会认为子类是一种父类类型。
            raise KeyError(key)
        return self[str(key)]
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    def __contains__(self, key):
        # 这种查询方式在 Python3 中很有效率，因为 self.keys() 返回的是一个 view，它和集合很像，速度和字典一样快
        # Python2 中返回的是列表，效率低
        # 这里使用 key in self.keys() 是有必要的，因为我们类中没有强迫字典所有的键都必须是 str 类型
        return key in self.keys() or str(key) in self.keys()
    
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
d['2']

'two'

In [17]:
d[4]

'four'

In [18]:
d[1]

KeyError: '1'

In [19]:
d.get('2')

'two'

In [20]:
d.get(4)

'four'

In [21]:
d.get(1, 'N/A')

'N/A'

In [22]:
2 in d

False

In [23]:
1 in d

False

## Dict 的变化

- collections.OrderedDict
  - 让键维持插入顺序， popitem 移除第一个元素，如果你使用 my_odict，popitem(last=True)调用，会移除最后一个元素
- collections.ChainMap
  - 保存一个可被当成单一元素搜寻的序列，查找时，会执行每一个映射，如果在它们之间找到键，成功。
  - 一个使用范例:
    ```python
    import builtins
    pylookup = ChainMap(locals(), globals(), vars(builtins))
    ```
- collections.Counter
  - 保存所有键的整数数量映射，更新会增加现有键的数量，这可以计算每个元素出现次数
  - 一个计算单词内字母数量的案例:
    ```python
    import collections
    ct = collections.Counter('abracadabra')
    ct
    #Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})
    ct.update('aaaaazzz')
    ct
    #Counter({'a': 15, 'b': 2, 'c': 1, 'd': 1, 'r': 2, 'z': 6})
    ct.most_common(2)
    #[('a', 15), ('z', 6)]
    ```
    
## 继承 UserDict

要建立新的映射类型，继承 UserDict 要比继承 dict 容易，因为它内部有一些很方便的运作方法，我们可以直接继承 UserDict 语法，不会出问题。我们要将前面的 StrKeyDict0 类所有的键都存成 str 类型

注意 UserDict 没有继承dict，而是内部有一个 dict 实例，称为 data，它会保存实际的元素，在编写 `__setitem__` 这种方法时，可以避免不受欢迎的回调。而且可以简化 `__contains__` 编写

下面除了将所有键存成 str 类型，当有人使用非字符串键构建或访问实例时，可以避免令人不愉快的意外

In [29]:
import collections

class StrKeyDict(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

因为 UserDict 是 MutableMapping 的子类，让 StrKeyDict 类的健全的映射方法都是从 UserDict, MutableMapping, Mapping 继承来的，下面是它比较优秀的方法:

- MutableMapping.update:
  - 我们可以直接调用这个方法，也可以通过 `__init__()` 调用，用来加载其它映射，可迭代（key, value）的实例，以及关键字个数。因为它使用 self[key] = value 来添加元素，最终调用的是 `__setitem__()`
  
- Mapping.get
  - 在 StrKeyDict0 中，我们必须自己编写 get 方法，这里我们继承 Mapping.get()，它的运作方式和 StrKeyDict0.get() 很像。
  
## 不可变映射

Python3.3 后，types 提供 MappingProxyType 包装类，当你给它一个映射后，它返回一个只读的 mappingproxy 实例，但它是原始映射的动态展示，也就是说，你可以在 mappingproxy 中看到原是映射的改变，但是无法通过它改变原始映射。下面的例子说明了这点

In [30]:
from types import MappingProxyType
d = {1: 'A'}
d_proxy = MappingProxyType(d)
d_proxy

mappingproxy({1: 'A'})

In [31]:
d_proxy[1]

'A'

In [32]:
d_proxy[2] = 'x'

TypeError: 'mappingproxy' object does not support item assignment

In [34]:
d[2] = 'b'
d_proxy

mappingproxy({1: 'A', 2: 'b'})

## 集合理论

我们说的集合包括 frozenset 和 set

In [1]:
l = ['spam', 'spam', 'eggs', 'spam']
set(l)

{'eggs', 'spam'}

In [2]:
list(set(l))

['spam', 'eggs']

集合的元素必须可哈希化，set 形态不可以哈希化，但是 frozenset 可以，所以可以在 set 中放入 frozenset 元素。

集合也支持基本的运算 a | b 代表 a 和 b 的并集， a & b 代表 a 和 b 的交集，a - b 为 a 和 b 的差集。

我们来看看计算 haystack 中的 needles 数量:

In [ ]:
#found = len(needles & haystack)

但是上面的语法只支持集合，我们让其支持所有可迭代类型

In [ ]:
#found = len(set(needles) & set(haystack))
# 也可以写成
#found = len(set(needles).intersection(haystack))

## set 常数

set 的常数定义语法（{1}, {1, 2} 等）看起来很像数学标记法，不过有一个重要的差别，空的 set 没有常数表示方法，必须显式的定义 set()

In [3]:
s = {1} #这种方式比 set([1]) 快很多，推荐使用
type(s)

set

In [4]:
s.pop()

1

In [5]:
s

set()

In [8]:
s = {}
type(s) #看到这样定义的是一个字典，而不是 set

dict

In [9]:
from dis import dis
dis('{1}')

  1           0 LOAD_CONST               0 (1)
              3 BUILD_SET                1
              6 RETURN_VALUE


In [10]:
dis('set([1])')

  1           0 LOAD_NAME                0 (set)
              3 LOAD_CONST               0 (1)
              6 BUILD_LIST               1
              9 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             12 RETURN_VALUE


看到 {1} 的方式直接执行了 1 的赋值， set([1]) 步骤比较多

frozenset 常数定义没有特殊的语法，必须使用构造式。

In [11]:
frozenset(range(10))

frozenset({0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

集合生成式和字典生成式差不多：

In [13]:
from unicodedata import name #用 name 函数获取字码名称有 'SIGN' 的子元集合
{chr(i) for i in range(32, 256) if 'SIGN' in name(chr(i), '')} # 构建字码名称有 'SIGN' 的子元集合

{'#',
 '$',
 '%',
 '+',
 '<',
 '=',
 '>',
 '¢',
 '£',
 '¤',
 '¥',
 '§',
 '©',
 '¬',
 '®',
 '°',
 '±',
 'µ',
 '¶',
 '×',
 '÷'}

## 集合运算

集合运算符要求两个运算元都是集合，但是其它所有方法并不是这样，例如要产生 a，b，c，d 的并集，a.union(b, c, d) 只要求 a 是集合，b，c，d 是可迭代对象就行

## 哈希值是否相等

如果两个元素比较是相等的，那么它们的哈希值一定相等，例如 1 == 1.0 为 True，所以 hash(1) == hash(1.0) 也是 True，就算 int 类型与 float 类型内部表示形式有很大差别（顺便说一句，int 类型的哈希值是 int 本身的值）

## dict 的实际效果

### 键必须是可哈希化元素

- 1. 可以由 hash() 得到一个不变的值。
- 2. 可以通过 eq() 方法判断是否相等
- 3. 如果 a == b 为 True，那么 hash(a) == hash(b) 为 True

如果你自定义了 eq()，一定要保证 a == b 和 hash(a) == hash(b) 结果相同，否则可能造成字典无法处理你的元素，报错

### dict 要使用大量的内存

### 用 key 搜索非常快

### 键的顺序取决与插入顺序

In [15]:
# DIAL_CODES = [
#     (86, 'China'),
#     (91, 'India'),
#     (1, 'United State')
# ]
# d1 = dict(DIAL_CODES)
# print('d1:', d1.keys())
# d2 = dict(sorted(DIAL_CODES))
# print('d2:', d2.keys())
# d3 = dict(sorted(DIAL_CODES, key = lambda x: x[1]))
# print('d3:', d3.keys())

### 将元素加到字典可能会改变现有键的顺序